# ComfyUI Playground (Runtime Python)

Minimal Colab notebook that installs and runs ComfyUI directly with the active Python runtime.
This version removes all virtual environment steps and focuses on the core workflow.
        

## Quick Start

1. (Optional) Run the Drive mount cell if you are inside Google Colab.
2. Configure paths and toggles in the configuration cell.
3. Run the setup cell to clone/update ComfyUI and install dependencies.
4. Launch ComfyUI.
5. Start the Cloudflare tunnel to obtain a public URL.

The notebook never creates or activates a virtual environment.
        

In [ ]:
#@title Configure paths and options
from pathlib import Path

REPO_URL = "https://github.com/Comfy-Org/ComfyUI.git"  #@param {type:"string"}
BRANCH = "master"  #@param {type:"string"}
DRIVE_ROOT = Path('/content/drive/MyDrive/ComfyUI')  # Persistent ComfyUI directory
INSTALL_REQUIREMENTS = True  #@param {type:"boolean"}
UPGRADE_BASE_PACKAGES = True  #@param {type:"boolean"}
PRINT_GIT_STATUS = True  #@param {type:"boolean"}

print('Repository URL:', REPO_URL)
print('Branch:', BRANCH)
print('Drive root:', DRIVE_ROOT)
        

In [ ]:
#@title Mount Google Drive (Colab only)
import os

if os.environ.get('COLAB_RELEASE_TAG'):
    from google.colab import drive
    if not os.path.ismount('/content/drive'):
        drive.mount('/content/drive')
    else:
        print('Drive already mounted.')
else:
    print('Google Colab not detected; skipping drive mount.')
        

In [ ]:
#@title Clone/Update ComfyUI and install dependencies
import os
import shutil
import subprocess
import sys
from pathlib import Path

def run_cmd(cmd, cwd=None):
    print('>>', ' '.join(str(c) for c in cmd))
    subprocess.run(cmd, check=True, cwd=cwd)

DRIVE_ROOT.mkdir(parents=True, exist_ok=True)

if not (DRIVE_ROOT / '.git').exists():
    print('Cloning ComfyUI into', DRIVE_ROOT)
    run_cmd(['git', 'clone', '--branch', BRANCH, REPO_URL, str(DRIVE_ROOT)])
else:
    print('Updating existing repository in', DRIVE_ROOT)
    run_cmd(['git', 'fetch', '--all'], cwd=DRIVE_ROOT)
    run_cmd(['git', 'reset', '--hard', f'origin/{BRANCH}'], cwd=DRIVE_ROOT)
    if PRINT_GIT_STATUS:
        run_cmd(['git', 'status', '-s'], cwd=DRIVE_ROOT)

if UPGRADE_BASE_PACKAGES:
    run_cmd([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip', 'setuptools', 'wheel'])

req_file = DRIVE_ROOT / 'requirements.txt'
if INSTALL_REQUIREMENTS:
    if req_file.exists():
        run_cmd([sys.executable, '-m', 'pip', 'install', '--upgrade', '-r', str(req_file)])
    else:
        print('No requirements.txt found at', req_file)
else:
    print('Dependency install skipped by user setting.')

print('Setup complete.')
        

In [ ]:
#@title Launch ComfyUI (headless)
import subprocess
import sys
import time
from pathlib import Path

COMFY_DIR = DRIVE_ROOT
COMMAND = [sys.executable, str(COMFY_DIR / 'main.py'), '--headless', '--port', '8188']

if not (COMFY_DIR / 'main.py').exists():
    raise FileNotFoundError('main.py not found in ' + str(COMFY_DIR))

print('Launching ComfyUI with:', ' '.join(COMMAND))
process = subprocess.Popen(COMMAND, cwd=str(COMFY_DIR), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
time.sleep(2)
if process.poll() is None:
    print('ComfyUI starting... streaming initial logs below. Press stop (square) to end log streaming.')
    for _ in range(40):
        line = process.stdout.readline()
        if not line:
            break
        print(line.rstrip())
else:
    print('ComfyUI exited early with code', process.returncode)

print('If the server keeps running, leave this cell active to keep ComfyUI alive.')
        

In [ ]:
#@title Start Cloudflare tunnel for ComfyUI
import shutil
import socket
import subprocess
import sys
import threading
import time
from pathlib import Path

CLOUDFLARED_DEB = Path('/tmp/cloudflared-linux-amd64.deb')
CLOUDFLARED_URL = 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb'

def install_cloudflared():
    if shutil.which('cloudflared'):
        return
    print('Downloading cloudflared package...')
    subprocess.run(['wget', '-q', '-O', str(CLOUDFLARED_DEB), CLOUDFLARED_URL], check=True)
    print('Installing cloudflared (dpkg)...')
    result = subprocess.run(['dpkg', '-i', str(CLOUDFLARED_DEB)], capture_output=True, text=True)
    if result.returncode != 0:
        print(result.stdout.strip())
        print(result.stderr.strip())
        raise RuntimeError('cloudflared install via dpkg failed; resolve dependencies and retry.')
    if not shutil.which('cloudflared'):
        raise RuntimeError('cloudflared binary not detected after install.')

def wait_for_port(port, host='127.0.0.1', timeout=180):
    start = time.time()
    while time.time() - start < timeout:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.settimeout(1.0)
            if sock.connect_ex((host, port)) == 0:
                return True
        time.sleep(0.5)
    return False

def stream_tunnel(port):
    print('Launching cloudflared tunnel for http://127.0.0.1:{} ...'.format(port))
    proc = subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    public_url = None
    for line in proc.stdout:
        line = line.strip()
        if line:
            print(line)
        if 'trycloudflare.com' in line:
            start = line.find('http')
            if start != -1:
                public_url = line[start:]
                print('Public URL:', public_url)
                break
    if public_url:
        print('Cloudflare tunnel ready. Leave this process running to keep the tunnel alive.')
    else:
        print('Could not detect public URL from cloudflared output. Check logs above.')

install_cloudflared()

requirements_path = DRIVE_ROOT / 'requirements.txt'
if requirements_path.exists():
    print('Ensuring runtime requirements are installed...')
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', str(requirements_path)], check=True)
else:
    print('No requirements.txt found; assuming dependencies were installed earlier.')

if wait_for_port(8188):
    print('ComfyUI port is reachable. Starting tunnel...')
else:
    print('ComfyUI port 8188 is not yet open; tunnel will still attempt to start.')

thread = threading.Thread(target=stream_tunnel, args=(8188,), daemon=True)
thread.start()
print('Background thread launched. Watch the output above for the public URL.')
        

In [ ]:
#@title Stop cloudflared and ComfyUI (cleanup)
import subprocess

subprocess.run(['pkill', '-f', 'cloudflared'], check=False)
subprocess.run(['pkill', '-f', 'main.py'], check=False)
print('Termination signals sent. Ignore "process not found" warnings if services were already stopped.')
        